<a href="https://colab.research.google.com/github/hongqin/AI4Health/blob/main/RAG_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Tutorial: Chat with a  Document using an Open-Source LLM on Google Colab


In [1]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.0 MB/s eta 0:00:00


In [2]:

# Step 1: Install Required Libraries
# To get started, we'll use the `langchain` library for building language model applications and the `transformers` library for loading open-source LLMs.

# Install libraries
!pip install langchain transformers sentence_transformers chromadb beautifulsoup4 requests protobuf==3.20.3

# Step 2: Load the Open-Source Language Model
# We'll use a pre-trained model like `tiiuae/falcon-7b` or similar. Make sure to select a model suitable for your needs.

from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Load model and tokenizer
def load_model():
    model_name = "tiiuae/falcon-7b"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")
    text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    llm = HuggingFacePipeline(pipeline=text_generator)
    return llm

llm = load_model()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.5 MB/s eta 0:00:00
 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-2-53913314bd2b>:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generator)


In [3]:
# Step 3: Fetch and Index Text from Wikipedia
# We'll scrape text from a Wikipedia page, process it, and index it for querying.

import requests
from bs4 import BeautifulSoup
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Fetch text from a Wikipedia page
def fetch_wikipedia_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = "".join([para.get_text() for para in paragraphs])
    return text

# Index the document
def index_document_from_text(text):
    # Split text into manageable chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_text(text)

    # Use embeddings for indexing
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    db = Chroma.from_texts(docs, embeddings)
    return db

# Example: Fetch and index a Wikipedia page
wiki_url = "https://en.wikipedia.org/wiki/Computer#History"
wikipedia_text = fetch_wikipedia_text(wiki_url)
db = index_document_from_text(wikipedia_text)


<ipython-input-3-7e8dc45ed84e>:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Step 4: Build a Chat Interface
# Create a function to interact with your document using the LLM and the indexed data.

from langchain.chains import RetrievalQA

# Define a query-answering chain
def create_qa_chain(llm, db):
    retriever = db.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa_chain

qa_chain = create_qa_chain(llm, db)

In [11]:
## add time

import time

# Step 5: Query the Document
# Finally, input a question and get an answer based on your document.

# Ask a question
def ask_question(question, max_response_length=50):
    print(f"(ask_question) Processing the question: {question}")
    start_time = time.time()

    response = qa_chain.run( {"query": question, "max_length":max_response_length})
    end_time = time.time()
    print(f"(ask_question) Question processed in {end_time - start_time:.2f} seconds.")
    print("(ask_question) Answer received successfully.")
    return response

question = "What is the history of computers?"
print("(main) Asking the question to the model...")
answer = ask_question(question)
print("(main) Answer:", answer)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


(main) Asking the question to the model...
(ask_question) Processing the question: What is the history of computers?


KeyboardInterrupt: 

In [8]:
import time

# Step 5: Query the Document
# Finally, input a question and get an answer based on your document.

# Ask a question
def ask_question(question, max_response_length=50):
    print(f"(ask_question) Processing the question: {question}")
    start_time = time.time()
    # The change is in the following line:
    # Instead of passing max_length as a keyword argument to qa_chain.run(),
    # we are setting the chain_type_kwargs to control the response length.
    response = qa_chain.run(question, chain_type_kwargs={"max_length": max_response_length})
    end_time = time.time()
    print(f"(ask_question) Question processed in {end_time - start_time:.2f} seconds.")
    print("(ask_question) Answer received successfully.")
    return response

question = "What is the history of computers?"
print("(main) Asking the question to the model...")
answer = ask_question(question)
print("(main) Answer:", answer)

(main) Asking the question to the model...
(ask_question) Processing the question: What is the history of computers?


ValueError: `run` supported with either positional arguments or keyword arguments but not both. Got args: ('What is the history of computers?',) and kwargs: {'chain_type_kwargs': {'max_length': 50}}.

In [6]:
# Step 5: Query the Document
# Finally, input a question and get an answer based on your document.

# Ask a question
def ask_question(question):
    print("(ask_questions):", question)
    response = qa_chain.run(question)
    return response

question = "What is the history of computers?"
answer = ask_question(question)
print("Answer:", answer)

# Step 6: Optional Enhancements
# - **Streamlit UI:** Build a web interface for interaction.
# - **Model Fine-Tuning:** Customize the model for your specific data.
# - **Cloud Integration:** Store and index large documents using cloud storage.

# This completes the tutorial on using an open-source LLM to chat with a Wikipedia page on Google Colab.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


(ask_questions): What is the history of computers?


KeyboardInterrupt: 